In [ ]:
#https://gist.github.com/jamescalam/7117aa92235a7f52141ad0654795aa48

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install faiss-gpu

In [7]:

import numpy as np
import faiss
import requests
from io import StringIO
import pandas as pd

In [8]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')

text = res.text
text[:100]


'pair_ID\tsentence_A\tsentence_B\trelatedness_score\tentailment_judgment\n1\tA group of kids is playing in '

In [9]:
data = pd.read_csv(StringIO(text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [10]:
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [11]:

sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)
len(set(sentences))

4802

In [12]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [13]:

for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())


b'Skipping line 191: expected 3 fields, saw 4\nSkipping line 206: expected 3 fields, saw 4\nSkipping line 295: expected 3 fields, saw 4\nSkipping line 695: expected 3 fields, saw 4\nSkipping line 699: expected 3 fields, saw 4\n'
b'Skipping line 104: expected 3 fields, saw 4\nSkipping line 181: expected 3 fields, saw 4\nSkipping line 317: expected 3 fields, saw 4\nSkipping line 412: expected 3 fields, saw 5\nSkipping line 508: expected 3 fields, saw 4\n'


In [14]:
len(set(sentences))

14505

In [62]:

# remove duplicates and NaN
sentences = [
    sentence.replace('\n', '') for sentence in list(set(sentences)) if type(sentence) is str
    ]

In [58]:
import nltk

In [63]:
sentences = [sentence.lower() for sentence in  sentences]

In [16]:
with open('sentences.txt', 'w') as fp:
    fp.write('\n'.join(sentences))

In [61]:
sentences = ['i love cricket',
             'i love cricket',
             'i love cricket',
             'i love cricket',
             'i love cricket',
             'i love cricket',
             'I love cricket and F1',
             'I love cricket and F1',
             'I love cricket and football',
             'I love cricket and Football',]

In [64]:
sentences


['i love cricket and football',
 'i love cricket and football',
 'i love cricket',
 'i love cricket and f1']

In [65]:

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

sentence_embeddings = model.encode(sentences[:20])
sentence_embeddings.shape

(4, 768)

In [22]:
with open(f'embeddings_X.npy', 'wb') as fp:
    np.save(fp, sentence_embeddings)

In [24]:
# saving data
split = 2
file_count = 0
for i in range(0, sentence_embeddings.shape[0], split):
    end = i + split
    if end > sentence_embeddings.shape[0] + 1:
        end = sentence_embeddings.shape[0] + 1
    file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
    with open(f'embeddings_{file_count}.npy', 'wb') as fp:
        np.save(fp, sentence_embeddings[i:end, :])
    print(f"embeddings_{file_count}.npy | {i} -> {end}")
    file_count = int(file_count) + 1

embeddings_0.npy | 0 -> 2
embeddings_1.npy | 2 -> 4
embeddings_2.npy | 4 -> 6
embeddings_3.npy | 6 -> 8
embeddings_4.npy | 8 -> 10
embeddings_5.npy | 10 -> 12
embeddings_6.npy | 12 -> 14
embeddings_7.npy | 14 -> 16
embeddings_8.npy | 16 -> 18
embeddings_9.npy | 18 -> 20


In [66]:
d = sentence_embeddings.shape[1]
d

768

In [67]:
index = faiss.IndexFlatL2(d)

In [51]:
index.is_trained

True

In [68]:
index.add(sentence_embeddings)

In [53]:
index.ntotal

4

In [69]:
k = 4
xq = model.encode(["i love cricket"])

In [70]:

%%time
D, I = index.search(xq, k)  # search
print(I)  # k-nearest neigbors of the query vector | nprobe == 1: 6495 26392 61709 49932 | nprobe == 10: 36245  6495 57489  8705

[[2 3 1 0]]
CPU times: user 1.83 ms, sys: 0 ns, total: 1.83 ms
Wall time: 989 µs


In [71]:
[f'{i}: {sentences[i]}' for i in I[0]]

['2: i love cricket',
 '3: i love cricket and f1',
 '1: i love cricket and football',
 '0: i love cricket and football']